In [12]:
spark.streams.active

In [1]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from pyspark.ml.evaluation import RegressionEvaluator

Using TensorFlow backend.


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
userSchema = StructType() \
        .add("window",StructType()\
             .add("start",TimestampType())\
             .add("end",TimestampType()))\
        .add("system", StringType())\
        .add("count", LongType())

In [4]:
raw_data = spark\
.readStream.format("parquet")\
.schema(userSchema)\
.load("/cms/users/carizapo/ming/groupdata_cmsweb_logs");
raw_data.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- system: string (nullable = true)
 |-- count: long (nullable = true)



In [5]:
raw_data_match = raw_data.withColumn('date',col("window.start")).drop(col("window"))

In [6]:
raw_data_flow = raw_data.writeStream.queryName("hdfs").outputMode("Append").format("memory").start()

In [10]:
raw_data_flow.lastProgress

{'id': '62892391-6575-4b53-8cec-b83aa6b414b6',
 'runId': '786e9e90-b5d6-4af9-81bc-58aad3a22370',
 'name': 'hdfs',
 'timestamp': '2019-07-11T13:44:36.798Z',
 'batchId': 1,
 'numInputRows': 0,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'getOffset': 788, 'triggerExecution': 788},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[hdfs://analytix/cms/users/carizapo/ming/groupdata_cmsweb_logs]',
   'startOffset': {'logOffset': 0},
   'endOffset': {'logOffset': 0},
   'numInputRows': 0,
   'inputRowsPerSecond': 0.0,
   'processedRowsPerSecond': 0.0}],
 'sink': {'description': 'MemorySink'}}

In [ ]:
raw_data_flow.stop()

In [11]:
alerts = spark.sql("select * from hdfs")
# alerts.show()

<li>Must renamed column in joining dataframe.</li>
<li>Generate unique id to every system. In order to do that we must apply window spec(work for static dataframe only) to the dataframe</li>

In [12]:
from pyspark.sql.window import Window as W
raw_data_id = alerts.select("system").distinct()
windowSpec = W.orderBy("system")
raw_data_id=raw_data_id.withColumn("systemID", row_number().over(windowSpec)).withColumnRenamed("system","tempSystem")

In [ ]:
raw_data_id.show()

In [13]:
alerts_df=alerts\
.join(raw_data_id, raw_data_id.tempSystem == alerts.system)\
.withColumn('date',col("window.start")).drop(col("window"))\
.drop(raw_data_id.tempSystem)

In [ ]:
alerts_df.show()
# raw_data_id.show()

In [ ]:
!pip install --user --upgrade dist-keras

In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler

In [15]:
%run StoreItemDemand/custom_transformers.ipynb

In [16]:
train_data, test_data = alerts_df.randomSplit([0.8,0.2], seed=1234)

In [17]:
df_train = train_data.withColumn('set', lit(0))
df_train = df_train.withColumn('id', lit(-1))
df_test = test_data.withColumn('set', lit(1))

df_test = df_test.withColumn('count', lit(-1))
joined = df_test.union(df_train.select(*df_test.columns))

train_data = joined.filter('set == 0')
test_data = joined.filter('set == 1')

In [18]:
train, validation = train_data.randomSplit([0.8,0.2], seed=1234)

In [19]:
SerieMaker?

In [20]:
# Feature extraction
dc = DateConverter(inputCol='date', outputCol='dateFormated')
dex = DayExtractor(inputCol='dateFormated')
mex = MonthExtractor(inputCol='dateFormated')
yex = YearExtractor(inputCol='dateFormated')
# Data process
#tentar fazer 'day', 'month', 'year', 'weekday', 'weekend' (as colunas derivadas) ficarem de forma dinâmica, no lugar delas ficar a saída de seu respectivo transformer
va = VectorAssembler(inputCols=['systemID','count', 'day', 'month', 'year'], outputCol="features")
# scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Serialize data
sm = SerieMaker(inputCol='scaledFeatures', dateCol='date', idCol=['systemID'], serieSize=30)

pipeline = Pipeline(stages=[dc, dex, mex, yex, va, scaler, sm])

In [21]:
pipiline_model = pipeline.fit(train)

In [22]:
train_transformed = pipiline_model.transform(train)
validation_transformed = pipiline_model.transform(validation)
test_transformed = pipiline_model.transform(test_data)

In [ ]:
# validation_stream=raw_data_match.join(validation_transformed, ["system","date","count"], "inner")
# test_stream=raw_data_match.join(test_transformed, ["system","date","count"], "inner")
# train_stream=raw_data_match.join(train_transformed, ["system","date","count"], "inner")

In [ ]:
# validation_stream_flow.stop()
# train_stream_flow.stop()
# test_stream_flow.stop()

In [ ]:
print('Train raw: %s' % train.count())
print('Validation raw: %s' % validation.count())
print('Test raw: %s' % test_data.count())

In [ ]:
print('Train transformed: %s' % train_transformed.count())
print('Validation transformed: %s' % validation_transformed.count())
print('Test transformed: %s' % test_transformed.count())

In [23]:
trainPd=train_transformed.select('serie', 'count').dropna().toPandas()

In [1]:
import numpy as np
from pyspark.ml.linalg import DenseVector,Vectors
train_try_1=trainPd
train_try_2=trainPd
a=Vectors.dense(np.array([0,0,0,0,0]))
# print(train_try_1['serie'].map(lambda item: item.map(lambda val: a if val is None else val)).head(20))
type(train_try_1['series'])
train_try_1['series'].head(20)

NameError: name 'trainPd' is not defined

In [ ]:
validationNp=validation_transformed.select('serie','count').dropna().toPandas()

In [ ]:
train_x = trainPd['serie'].values.tolist()
train_y = trainPd['count'].values.tolist()
validation_x = validationPd['serie'].values.tolist()
validation_y = validationPd['count'].values.tolist()

In [ ]:
train_x_np = np.array(train_x)
train_y_np = np.array(train_y)

In [ ]:
train_y.shape

In [ ]:
train_x.shape

In [ ]:
n_label = 1
serie_size = len(train_x_np[0])
n_features = len(train_x_np[0][0])

In [ ]:
[ if v is None else v for v in train_x_np]
train_x[10]

In [ ]:
train_y= [i if i[0] is not None else (0, i[1]) for i in train_y_np]

In [ ]:
from keras.layers import Input, Flatten, Reshape

In [ ]:
# hyperparameters
epochs = 80
batch = 512
lr = 0.001

# design network
model = Sequential()
model.add(GRU(40, input_shape=(serie_size, n_features)))

model.add(Flatten())
model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(n_label))

model.summary()

adam = optimizers.Adam(lr)
model.compile(loss='mae', optimizer=adam, metrics=['mse', 'msle'])

history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch, validation_data=(validation_x, validation_y), verbose=2, shuffle=False)